## 1. Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded successfully!")

In [ ]:
# Load merged dataset
df = pd.read_csv('../data/merged_pos_data.csv', parse_dates=['Date', 'ManufacturingDate', 'ExpiryDate'])

print("Dataset Loaded!")
print(f"Total Records: {len(df):,}")
print(f"Columns: {df.shape[1]}")
print(f"Date Range: {df['Date'].min().date()} to {df['Date'].max().date()}")

## 2. Data Overview

In [ ]:
# Display first few rows
print("Sample Data:")
df.head(10)

In [ ]:
# Data types and info
print("Dataset Information:")
df.info()

In [ ]:
# Statistical summary
print("Statistical Summary:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✓ No missing values detected!")
else:
    print(missing[missing > 0])

## 3. Category Analysis

In [ ]:
# Category distribution
print("Category Distribution:")
print("="*60)

cat_summary = df.groupby('Category').agg({
    'TransactionID': 'count',
    'NetAmount': ['sum', 'mean'],
    'Profit': 'sum',
    'MarginPercent': 'mean',
    'SKU_ID': 'nunique'
}).round(2)

cat_summary.columns = ['Transactions', 'Total_Revenue', 'Avg_Revenue', 'Total_Profit', 'Avg_Margin', 'Unique_SKUs']
print(cat_summary)

In [ ]:
# Visualize category performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Revenue by category
cat_revenue = df.groupby('Category')['NetAmount'].sum() / 1e6
axes[0].bar(cat_revenue.index, cat_revenue.values, color=['#2E86AB', '#A23B72'], edgecolor='black', linewidth=1.5)
axes[0].set_title('Revenue by Category', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Revenue (Million INR)', fontweight='bold')
for i, v in enumerate(cat_revenue.values):
    axes[0].text(i, v + 10, f'₹{v:.1f}M', ha='center', fontweight='bold')

# Margin by category
cat_margin = df.groupby('Category')['MarginPercent'].mean()
axes[1].bar(cat_margin.index, cat_margin.values, color=['#2E86AB', '#A23B72'], edgecolor='black', linewidth=1.5)
axes[1].set_title('Average Margin % by Category', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Margin %', fontweight='bold')
for i, v in enumerate(cat_margin.values):
    axes[1].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. SubCategory Analysis

In [ ]:
# SubCategory performance
print("SubCategory Performance:")
print("="*60)

subcat_summary = df.groupby(['Category', 'SubCategory']).agg({
    'TransactionID': 'count',
    'NetAmount': 'sum',
    'MarginPercent': 'mean'
}).round(2)

subcat_summary.columns = ['Transactions', 'Total_Revenue', 'Avg_Margin']
subcat_summary = subcat_summary.sort_values('Total_Revenue', ascending=False)
print(subcat_summary)

## 5. POS System Comparison

In [ ]:
# POS system comparison
print("POS System Comparison:")
print("="*60)

pos_summary = df.groupby('POS_System').agg({
    'TransactionID': 'count',
    'NetAmount': ['sum', 'mean'],
    'Profit': 'sum',
    'MarginPercent': 'mean',
    'Store': 'nunique'
}).round(2)

pos_summary.columns = ['Transactions', 'Total_Revenue', 'Avg_Revenue', 'Total_Profit', 'Avg_Margin', 'Stores']
print(pos_summary)

print("\nStore-Level Performance:")
print("="*60)
store_summary = df.groupby(['POS_System', 'Store']).agg({
    'NetAmount': 'sum',
    'TransactionID': 'count'
}).round(2)
store_summary.columns = ['Total_Revenue', 'Transactions']
print(store_summary)

In [ ]:
# Visualize POS comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Revenue by POS
pos_revenue = df.groupby('POS_System')['NetAmount'].sum() / 1e6
axes[0].bar(pos_revenue.index, pos_revenue.values, color=['#FF6B6B', '#4ECDC4'], edgecolor='black', linewidth=1.5)
axes[0].set_title('Revenue by POS System', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Revenue (Million INR)', fontweight='bold')
for i, v in enumerate(pos_revenue.values):
    axes[0].text(i, v + 10, f'₹{v:.1f}M', ha='center', fontweight='bold')

# Transactions by POS
pos_trans = df.groupby('POS_System')['TransactionID'].count() / 1000
axes[1].bar(pos_trans.index, pos_trans.values, color=['#FF6B6B', '#4ECDC4'], edgecolor='black', linewidth=1.5)
axes[1].set_title('Transactions by POS System', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Transactions (Thousands)', fontweight='bold')
for i, v in enumerate(pos_trans.values):
    axes[1].text(i, v + 1, f'{v:.1f}K', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Stock Ageing Analysis

In [ ]:
# Stock ageing distribution
print("Stock Ageing Analysis:")
print("="*60)

age_summary = df.groupby('StockAgeCategory').agg({
    'TransactionID': 'count',
    'NetAmount': 'sum',
    'StockAgeDays': 'mean'
}).round(2)

age_summary.columns = ['Transactions', 'Total_Revenue', 'Avg_Stock_Age']
age_summary['Percentage'] = (age_summary['Transactions'] / len(df) * 100).round(2)
print(age_summary)

print("\nStock Ageing by Category:")
print("="*60)
age_by_cat = df.groupby(['Category', 'StockAgeCategory']).size().unstack(fill_value=0)
age_by_cat_pct = (age_by_cat.div(age_by_cat.sum(axis=1), axis=0) * 100).round(2)
print(age_by_cat_pct)

In [ ]:
# Visualize stock ageing
age_counts = df['StockAgeCategory'].value_counts()
colors = ['#2ECC71', '#3498DB', '#F39C12', '#E74C3C']

plt.figure(figsize=(8, 6))
plt.pie(age_counts.values, labels=age_counts.index, autopct='%1.1f%%', colors=colors[:len(age_counts)], 
        startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
plt.title('Stock Ageing Distribution', fontweight='bold', fontsize=14)
plt.show()

## 7. Margin Analysis

In [ ]:
# Margin analysis
print("Margin Analysis:")
print("="*60)

print(f"Overall Average Margin: {df['MarginPercent'].mean():.2f}%")
print(f"Minimum Margin: {df['MarginPercent'].min():.2f}%")
print(f"Maximum Margin: {df['MarginPercent'].max():.2f}%")
print(f"Median Margin: {df['MarginPercent'].median():.2f}%")

print("\nMargin by Category & SubCategory:")
print("="*60)
margin_summary = df.groupby(['Category', 'SubCategory'])['MarginPercent'].agg(['mean', 'min', 'max']).round(2)
margin_summary.columns = ['Avg_Margin', 'Min_Margin', 'Max_Margin']
print(margin_summary.sort_values('Avg_Margin', ascending=False))

In [ ]:
# Margin distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df[df['Category']=='Grocery']['MarginPercent'], bins=30, color='#2E86AB', alpha=0.7, edgecolor='black', label='Grocery')
plt.hist(df[df['Category']=='Fashion']['MarginPercent'], bins=30, color='#A23B72', alpha=0.7, edgecolor='black', label='Fashion')
plt.xlabel('Margin %', fontweight='bold')
plt.ylabel('Frequency', fontweight='bold')
plt.title('Margin Distribution by Category', fontweight='bold', fontsize=12)
plt.legend()

plt.subplot(1, 2, 2)
df.boxplot(column='MarginPercent', by='Category', ax=plt.gca(), patch_artist=True)
plt.title('Margin Distribution by Category', fontweight='bold', fontsize=12)
plt.suptitle('')
plt.xlabel('Category', fontweight='bold')
plt.ylabel('Margin %', fontweight='bold')

plt.tight_layout()
plt.show()

## 8. Temporal Analysis

In [ ]:
# Monthly trends
df['YearMonth'] = df['Date'].dt.to_period('M')

monthly_revenue = df.groupby(['YearMonth', 'Category'])['NetAmount'].sum().unstack() / 1e6
monthly_trans = df.groupby(['YearMonth', 'Category'])['TransactionID'].count().unstack() / 1000

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Revenue trends
monthly_revenue.plot(ax=axes[0], marker='o', linewidth=2, color=['#2E86AB', '#A23B72'])
axes[0].set_title('Monthly Revenue Trends', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Revenue (Million INR)', fontweight='bold')
axes[0].set_xlabel('Month', fontweight='bold')
axes[0].legend(title='Category')
axes[0].grid(True, alpha=0.3)

# Transaction trends
monthly_trans.plot(ax=axes[1], marker='s', linewidth=2, color=['#2E86AB', '#A23B72'])
axes[1].set_title('Monthly Transaction Trends', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Transactions (Thousands)', fontweight='bold')
axes[1].set_xlabel('Month', fontweight='bold')
axes[1].legend(title='Category')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Weekend vs Weekday analysis
print("Weekend vs Weekday Performance:")
print("="*60)

weekend_summary = df.groupby(['IsWeekend', 'Category']).agg({
    'TransactionID': 'count',
    'NetAmount': ['sum', 'mean']
}).round(2)

weekend_summary.index = ['Weekday', 'Weekend']
print(weekend_summary)

## 9. Brand Analysis

In [ ]:
# Top brands
print("Top 10 Brands by Revenue:")
print("="*60)

top_brands = df.groupby('Brand').agg({
    'NetAmount': 'sum',
    'TransactionID': 'count',
    'MarginPercent': 'mean'
}).round(2)

top_brands.columns = ['Total_Revenue', 'Transactions', 'Avg_Margin']
top_brands = top_brands.sort_values('Total_Revenue', ascending=False).head(10)
print(top_brands)

## 10. Key Insights and Summary

In [ ]:
# Summary KPIs
print("="*60)
print("KEY PERFORMANCE INDICATORS")
print("="*60)
print()
print(f"Total Transactions: {len(df):,}")
print(f"Total Revenue: ₹{df['NetAmount'].sum()/1e6:.2f}M")
print(f"Total Profit: ₹{df['Profit'].sum()/1e6:.2f}M")
print(f"Average Margin: {df['MarginPercent'].mean():.2f}%")
print(f"Average Transaction Value: ₹{df['NetAmount'].mean():.2f}")
print(f"Unique SKUs: {df['SKU_ID'].nunique():,}")
print(f"Unique Stores: {df['Store'].nunique()}")
print()
print("Category Split:")
print(f"  Grocery: {len(df[df['Category']=='Grocery'])/len(df)*100:.1f}% of transactions")
print(f"  Fashion: {len(df[df['Category']=='Fashion'])/len(df)*100:.1f}% of transactions")
print()
print("POS Split:")
print(f"  POS1: {len(df[df['POS_System']=='POS1'])/len(df)*100:.1f}% of transactions")
print(f"  POS2: {len(df[df['POS_System']=='POS2'])/len(df)*100:.1f}% of transactions")
print()
print("Stock Health:")
print(f"  Fresh Stock: {len(df[df['StockAgeCategory']=='Fresh'])/len(df)*100:.1f}%")
print(f"  Ageing Stock: {len(df[df['StockAgeCategory']=='Ageing'])/len(df)*100:.1f}%")
print()
print("="*60)

## Conclusions

**Key Findings:**

1. **Category Performance:** Fashion generates higher revenue per transaction but Grocery has higher transaction volume
2. **Margin Analysis:** Fashion category shows significantly higher margins (~45%) compared to Grocery (~15%)
3. **POS System:** Both POS1 and POS2 show similar performance patterns with balanced contribution
4. **Stock Management:** Majority of stock is in Normal/Ageing categories, indicating need for inventory optimization
5. **Temporal Patterns:** Weekend sales show higher transaction values compared to weekdays
6. **Brand Performance:** Top 5 brands contribute to significant portion of total revenue

**Recommendations:**
- Focus on stock rotation for ageing inventory
- Optimize margin strategies for Grocery category
- Leverage weekend traffic with targeted promotions
- Maintain strong performance of top brands while exploring opportunities in underperforming brands

---

*End of Analysis*